In [101]:
import logging
from itertools import combinations
import pandas as pd
import numpy as np
from geopy.distance import geodesic
import random

logging.basicConfig(level=logging.DEBUG)

# List of city files to analyze
city_files = ['cities/vanuatu.csv', 'cities/italy.csv', 'cities/us.csv', 'cities/russia.csv', 'cities/china.csv' ]  # Add paths to other city files if needed
# city_files = [ 'cities/vanuatu.csv']  # Add paths to other city files if needed


def load_cities(file_path):
    """Load city data from a CSV file and return it as a DataFrame."""
    return pd.read_csv(file_path, header=None, names=['name', 'lat', 'lon'])

def create_distance_matrix(cities):
    """Create a distance matrix for the given city DataFrame."""
    n = len(cities)
    dist_matrix = np.zeros((n, n))
    for c1, c2 in combinations(cities.itertuples(), 2):
        dist_matrix[c1.Index, c2.Index] = dist_matrix[c2.Index, c1.Index] = geodesic(
            (c1.lat, c1.lon), (c2.lat, c2.lon)
        ).km
    return dist_matrix


In [102]:
def tsp_greedy(dist_matrix, cities):
    """Solve TSP using a greedy nearest-neighbor approach and return the path and total cost."""
    n = dist_matrix.shape[0]
    path = [0]  # Start from the first city (index 0)
    visited = set(path)
    total_cost = 0.0
    current_city = 0

    for _ in range(n - 1):
        next_city = None
        min_distance = float('inf')
        for j in range(n):
            if j not in visited and dist_matrix[current_city, j] < min_distance:
                min_distance = dist_matrix[current_city, j]
                next_city = j

        path.append(next_city)
        visited.add(next_city)
        total_cost += min_distance

        # Log each step
        '''logging.debug(
            f"Step: {cities.at[current_city, 'name']} -> {cities.at[next_city, 'name']} ({dist_matrix[current_city, next_city]:.2f} km)"
        )'''

        current_city = next_city

    # Return to the starting city and update the cost
    total_cost += dist_matrix[current_city, path[0]]
    path.append(path[0])  # Close the loop by returning to the starting city

    # Log final step
    '''logging.debug(
        f"Step: {cities.at[current_city, 'name']} -> {cities.at[path[0], 'name']} ({dist_matrix[current_city, path[0]]:.2f} km)"
    )'''

    return path, total_cost

In [103]:

# Main execution
for file_path in sorted(city_files):
    # Load city data and create distance matrix
    cities = load_cities(file_path)
    dist_matrix = create_distance_matrix(cities)

    # Solve TSP and get the path and total cost
    path, total_cost = tsp_greedy(dist_matrix, cities)

    # Print sorted results for each file
    print(f"\n=== Results for {file_path.split('/')[-1]} ===")
    '''for i in range(len(path) - 1):
        print(f"{cities.at[path[i], 'name']} -> {cities.at[path[i+1], 'name']} ({dist_matrix[path[i], path[i+1]]:.2f} km)")'''
    print(f"Total Path Cost: {total_cost:.2f} km\n")



=== Results for china.csv ===
Total Path Cost: 63962.92 km


=== Results for italy.csv ===
Total Path Cost: 4436.03 km


=== Results for russia.csv ===
Total Path Cost: 42334.16 km


=== Results for us.csv ===
Total Path Cost: 48050.03 km


=== Results for vanuatu.csv ===
Total Path Cost: 1475.53 km



In [104]:
def hill_climber(dist_matrix, cities, iterazioni):
    """Algoritmo Hill Climber per risolvere il problema del TSP."""
    n = len(dist_matrix)
    # Inizializza un percorso casuale
    percorso = list(range(n))
    random.shuffle(percorso)
    
    def calcola_distanza_totale(percorso):
        return sum(dist_matrix[percorso[i], percorso[i + 1]] for i in range(n - 1)) + dist_matrix[percorso[-1], percorso[0]]

    # Calcola la distanza del percorso iniziale
    migliore_distanza = calcola_distanza_totale(percorso)
    logging.debug("Inizio percorso casuale: " + " -> ".join(cities.at[i, 'name'] for i in percorso))
    
    for _ in range(iterazioni):
        # Crea un nuovo percorso invertendo una sottosequenza (2-opt)
        nuovo_percorso = percorso[:]
        i, j = random.sample(range(1, n), 2)
        if i > j:
            i, j = j, i
        nuovo_percorso[i:j+1] = reversed(nuovo_percorso[i:j+1])
        
        # Calcola la distanza del nuovo percorso
        nuova_distanza = calcola_distanza_totale(nuovo_percorso)
        
        # Se è migliore, aggiorna il percorso
        if nuova_distanza < migliore_distanza:
            percorso, migliore_distanza = nuovo_percorso, nuova_distanza
            #logging.debug(f"Nuova distanza trovata con Hill Climber: {migliore_distanza:.2f} km")
            # Log di ogni step nel nuovo percorso
            for k in range(n - 1):
                current_city = percorso[k]
                next_city = percorso[k + 1]
                '''logging.debug(
                    f"Step: {cities.at[current_city, 'name']} -> {cities.at[next_city, 'name']} "
                    f"({dist_matrix[current_city, next_city]:.2f} km)"
                )'''
            # Ultimo step per chiudere il ciclo
            '''logging.debug(
                f"Step: {cities.at[percorso[-1], 'name']} -> {cities.at[percorso[0], 'name']} "
                f"({dist_matrix[percorso[-1], percorso[0]]:.2f} km)"
            )'''
    
    return percorso, migliore_distanza


In [105]:
import math
import random

def simulated_annealing(dist_matrix, cities, iterazioni, temp_iniziale, temp_finale, fattore_raffreddamento):
    """Algoritmo Simulated Annealing per risolvere il problema del TSP."""
    n = len(dist_matrix)
    percorso = list(range(n))
    random.shuffle(percorso)

    def calcola_distanza_totale(percorso):
        return sum(dist_matrix[percorso[i], percorso[i + 1]] for i in range(n - 1)) + dist_matrix[percorso[-1], percorso[0]]

    distanza_corrente = calcola_distanza_totale(percorso)
    migliore_distanza = distanza_corrente
    migliore_percorso = percorso[:]
    
    temperatura = temp_iniziale
    
    while temperatura > temp_finale:
        for _ in range(iterazioni):
            nuovo_percorso = percorso[:]
            i, j = sorted(random.sample(range(n), 2))
            nuovo_percorso[i:j+1] = reversed(nuovo_percorso[i:j+1])
            nuova_distanza = calcola_distanza_totale(nuovo_percorso)
            delta_distanza = nuova_distanza - distanza_corrente

            # Condizione per accettare il percorso
            if delta_distanza < 0 or random.random() < math.exp(-delta_distanza / temperatura):
                percorso, distanza_corrente = nuovo_percorso, nuova_distanza
                if distanza_corrente < migliore_distanza:
                    migliore_percorso, migliore_distanza = percorso[:], distanza_corrente
                    #logging.debug(f"Nuovo percorso ottimizzato a {migliore_distanza:.2f} km")

        # Raffreddamento
        temperatura *= fattore_raffreddamento

    # Log finale del percorso migliore
    # logging.debug("Miglior percorso trovato:")
    # for k in range(n - 1):
    #     logging.debug(
    #         f"{cities.at[migliore_percorso[k], 'name']} -> {cities.at[migliore_percorso[k + 1], 'name']} "
    #         f"({dist_matrix[migliore_percorso[k], migliore_percorso[k + 1]]:.2f} km)"
    #     )
    # logging.debug(
    #     f"{cities.at[migliore_percorso[-1], 'name']} -> {cities.at[migliore_percorso[0], 'name']} "
    #     f"({dist_matrix[migliore_percorso[-1], migliore_percorso[0]]:.2f} km)"
    # )
    
    return migliore_percorso, migliore_distanza


In [106]:
for file in city_files:
    cities = load_cities(file)
    dist_matrix = create_distance_matrix(cities)

    # Parametri
    iterazioni = 500#10000
    temp_iniziale = 100
    temp_finale = 0.01
    fattore_raffreddamento = 0.99

    # # Hill Climber
    # logging.info("Esecuzione dell'algoritmo Hill Climber...")
    # percorso_hc, distanza_hc = hill_climber(dist_matrix,cities, iterazioni)
    # print(f"Percorso migliore con Hill Climber: , Distanza: {distanza_hc:.2f} km")#{percorso_hc}

    # Simulated Annealing
    logging.info("Esecuzione dell'algoritmo Simulated Annealing...")
    #dist_matrix, cities, iterazioni, temp_iniziale, temp_finale, fattore_raffreddamento
    percorso_sa, distanza_sa = simulated_annealing(dist_matrix,cities, iterazioni, temp_iniziale, temp_finale, fattore_raffreddamento)
    print(f"Percorso migliore con Simulated Annealing: , Distanza: {distanza_sa:.2f} km")#{percorso_sa}

INFO:root:Esecuzione dell'algoritmo Simulated Annealing...


Percorso migliore con Simulated Annealing: , Distanza: 1345.54 km


INFO:root:Esecuzione dell'algoritmo Simulated Annealing...


Percorso migliore con Simulated Annealing: , Distanza: 4174.94 km


INFO:root:Esecuzione dell'algoritmo Simulated Annealing...


Percorso migliore con Simulated Annealing: , Distanza: 40665.66 km


INFO:root:Esecuzione dell'algoritmo Simulated Annealing...


Percorso migliore con Simulated Annealing: , Distanza: 34037.59 km


INFO:root:Esecuzione dell'algoritmo Simulated Annealing...


Percorso migliore con Simulated Annealing: , Distanza: 62496.74 km


In [107]:
# def total_distance(tour, dist_matrix):
#     """Calculate the total distance of a given tour using the distance matrix."""
#     return sum(dist_matrix[tour[i], tour[i + 1]] for i in range(len(tour) - 1)) + dist_matrix[tour[-1], tour[0]]
# def generate_population(size, n_cities):
#     """Generate an initial population of random tours."""
#     return [random.sample(range(n_cities), n_cities) for _ in range(size)]
# def mutate(tour):
#     """Perform a random mutation in a tour using a 2-opt swap."""
#     idx1, idx2 = random.sample(range(len(tour)), 2)
#     tour[idx1], tour[idx2] = tour[idx2], tour[idx1]
#     return tour

# def two_opt(tour, dist_matrix):
#     """Refine a tour using the 2-opt swap method to reduce path cost."""
#     best = tour
#     best_distance = total_distance(tour, dist_matrix)
#     for i in range(len(tour) - 1):
#         for j in range(i + 2, len(tour)):
#             if j - i == 1: continue  # Skip consecutive cities
#             new_tour = tour[:i] + tour[i:j][::-1] + tour[j:]
#             new_distance = total_distance(new_tour, dist_matrix)
#             if new_distance < best_distance:
#                 best, best_distance = new_tour, new_distance
#     return best

# def genetic_algorithm(cities, dist_matrix, population_size=100, generations=1000, elitism_rate=0.1):
#     """Genetic algorithm to solve the TSP with improvements."""
#     # Generate initial population
#     population = generate_population(population_size, len(cities))
#     best_tour = min(population, key=lambda tour: total_distance(tour, dist_matrix))
#     best_distance = total_distance(best_tour, dist_matrix)
    
#     for generation in range(generations):
#         # Apply mutation to each individual, then apply 2-opt to improve each tour
#         mutated_population = [two_opt(mutate(tour[:]), dist_matrix) for tour in population]
        
#         # Add elitism: keep the top-performing individuals
#         num_elite = int(elitism_rate * population_size)
#         elite_population = sorted(mutated_population, key=lambda tour: total_distance(tour, dist_matrix))[:num_elite]
        
#         # Fill the rest of the population with new mutated individuals
#         population = elite_population + [mutate(random.choice(elite_population)[:]) for _ in range(population_size - num_elite)]
        
#         # Evaluate the best tour in this generation
#         current_best = min(population, key=lambda tour: total_distance(tour, dist_matrix))
#         current_distance = total_distance(current_best, dist_matrix)
        
#         # Update the best tour found
#         if current_distance < best_distance:
#             best_tour, best_distance = current_best, current_distance
#             logging.debug(f"Generation {generation}: New best distance found: {best_distance}")

#     # Log the best tour found
#     logging.debug("Best path found:")
#     for i in range(len(best_tour)):
#         current_city = best_tour[i]
#         next_city = best_tour[(i + 1) % len(best_tour)]
#         logging.debug(
#             f"Step: {cities.at[current_city, 'name']} -> {cities.at[next_city, 'name']} ({dist_matrix[current_city, next_city]:.2f} km)"
#         )

#     return best_tour, best_distance

# # Example usage
# for file in city_files:
#     cities = load_cities(file)
#     dist_matrix = create_distance_matrix(cities)
#     best_tour, best_distance = genetic_algorithm(cities, dist_matrix)
#     print(f"File: {file} - Best tour: {best_tour}, Best distance: {best_distance:.2f} km")